In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
from nltk import word_tokenize
from Levenshtein import distance
import re

In [7]:
# settings
pd.options.display.max_columns=1000
pd.options.display.max_colwidth=None
pd.options.display.max_rows=100

# path
temp_dir = Path('../src/templates/')
temp_dir.absolute()

PosixPath('/home/avyas/projects/project_off/digitrubber/template_analysis/../src/templates')

In [13]:
# jaccard distance
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

# list of closest labels
def measure_similarity(text1, text2):
    if type(text1) ==  str and type(text2) ==str:
        text1 = text1.strip().lower()
        text2 = re.sub('\[.*\]', '', text2).strip().lower()
        s1 = jaccard_similarity(word_tokenize(text1), word_tokenize(text2)) > 0.8
        s2 = distance(text1.lower(), text2.lower()) < 4
        return (s1, s2)
    else:
        return False

def measure_similarity1(text1, text2):
    if type(text1) ==  str and type(text2) ==str:
        text1 = text1.strip().lower()
        text2 = text2.strip().lower()
        s1 = jaccard_similarity(word_tokenize(text1), word_tokenize(text2)) > 0.8
        s2 = distance(text1.lower(), text2.lower()) < 4
        return (s1, s2)
    else:
        return False

def is_close_labels(text, text_l, func):
    try:
        # print (text, "  ############  " ,text_l)
        close_label_list = [i for i in text_l if any(func(text, i))]
        return False if len(close_label_list) > 0 else True
    except Exception as e:
        print('Error:', e)
        return True

def find_very_close_labels(text, text_l, func):
    try:
        close_label_list = [i for i in text_l if all(func(text, i))]
        return close_label_list if len(close_label_list) > 0 else np.nan
    except Exception as e:
        print('Error:', e)
        return True

def find_close_labels(text, text_l, func):
    try:
        close_label_list = [i for i in text_l if any(func(text, i))]
        return close_label_list if len(close_label_list) > 0 else np.nan
    except Exception as e:
        print('Error:', e)
        return True

def read_excel(excel_file):
    df_excel = pd.read_excel(excel_file, header=1).dropna(axis=0, how='all')
    df_excel = df_excel.dropna(axis=1, how='all')
    return df_excel

def read_csv(csv_file):
    df_csv = pd.read_csv(csv_file, encoding='utf-8').dropna(axis=0, how='all')
    df_csv = df_csv.dropna(axis=1, how='all')
    return df_csv
    

def add_close_labels(df_excel, df_csv, csv_close_label, label_excel='Label', label_csv='Label', func=measure_similarity, label_pos=1):
    # preprocessing files
    df_excel[label_excel] = df_excel[label_excel].fillna(method='ffill')
    df_excel['Missing'] = df_excel[label_excel]\
                              .apply(is_close_labels, args=(df_csv[label_csv].dropna(),func))
    
    df_excel['VeryCloseLabelStrings'] = df_excel[label_excel]\
                              .apply(find_very_close_labels, args=(df_csv[label_csv].dropna(),func))
    
    df_excel['CloseLabelStrings'] = df_excel[label_excel]\
                              .apply(find_close_labels, args=(df_csv[label_csv].dropna(),func))
    columns = df_excel.columns.tolist()
    df_excel = df_excel[columns[0:label_pos]+ columns[-3:]+columns[label_pos:-3]]

    df_excel.to_csv(csv_close_label, encoding='utf-8', index=False)


In [9]:
# excel and template files path
excel_file1 = './excels/dict_dik_v3.xlsx'
csv_file1 = './csvs/dik_template_16_09.csv' 
csv_close_label1 = './csvs_close_label/dik_v3_template_16_09.csv'
add_close_labels(read_excel(excel_file1), read_csv(csv_file1), csv_close_label1, 'Label [English]')

# excel and template files path
excel_file2 = './excels/dict_hsh_V2.xlsx'
csv_file2 = './csvs/robot_hsh_template_13_09.csv' 
csv_close_label2 = './csvs_close_label/hsh_v2_template_13_09.csv'
add_close_labels(excel_file2, csv_file2, csv_close_label2)

# excel and template files path
excel_file3 = './excels/dict_ifnano_V2.xlsx'
csv_file3 = './csvs/robot_ifnano_template_30_08.csv' 
csv_close_label3 = './csvs_close_label/infnano_v2_template_30_08.csv'
add_close_labels(excel_file3, csv_file3, csv_close_label3)

# excel and template files path
excel_file4 = './excels/dict_imr_V2.xlsx'
csv_file4 = './csvs/robot_imr_template_08_09.csv' 
csv_close_label4 = './csvs_close_label/imr_v2_template_08_09.csv'
add_close_labels(excel_file4, csv_file4, csv_close_label4)

# excel and template files path
excel_file5 = './excels/dict_ita_V2.xlsx'
csv_file5 = './csvs/robot_ita_template_02_06.csv' 
csv_close_label5 = './csvs_close_label/ita_v2_template_02_06.csv'
add_close_labels(excel_file5, csv_file5, csv_close_label5)

# excel and template files path
excel_file6 = './excels/dict_jade_V2.xlsx'
csv_file6 = './csvs/robot_jade_template_09_06.csv' 
csv_close_label6 = './csvs_close_label/jade_v2_template_09_06.csv'
add_close_labels(read_excel(excel_file6), read_csv(csv_file6), csv_close_label6)

In [10]:
# Merging all csvs by luis
df_csv1 = pd.read_csv(csv_file1, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv2 = pd.read_csv(csv_file2, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv3 = pd.read_csv(csv_file3, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv4 = pd.read_csv(csv_file4, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv5 = pd.read_csv(csv_file5, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_csv6 = pd.read_csv(csv_file5, encoding='utf-8').dropna(axis=0, how='all').dropna(axis=1, how='all')
df_merg = pd.concat([df_csv1, df_csv2, df_csv3, df_csv4, df_csv5, df_csv6], ignore_index=True)
df_merg.drop_duplicates(inplace=True)
df_merg.to_csv('./csvs/merged_csvs.csv', index=False, encoding='utf-8')
df_merg.sort_values(by=['SubClass', 'Label', 'Label.1'], inplace=True)
df_merg.to_csv('./csvs/merged_csvs_sorted.csv', index=False, encoding='utf-8')

In [14]:
merged_csv_akh = './csvs/merged_csvs_sorted.csv'
merged_csv_luis = './csvs/digitrubber_robot_v4.csv'
csv_close_label7 = './csvs_close_label/merged_csv_digitrubber_robot_v4.csv'
add_close_labels(read_csv(merged_csv_akh), read_csv(merged_csv_luis), csv_close_label7, func=measure_similarity1, label_pos=2)

In [17]:
read_excel(excel_file1).columns ,read_excel(excel_file2).columns, read_excel(excel_file3).columns, read_excel(excel_file4).columns, read_excel(excel_file5).columns, read_excel(excel_file6).columns

(Index(['Label [English]', 'Label [Deutsch]', 'Definition [English]',
        'Definition [Deutsch]', 'Parent Class', 'Denotation/symbol/acronym',
        'Synonyms [English]', 'Synonyms [Deutsch]', 'identifier',
        'Notes [English]', 'Notes [Deutsch]', 'Example of usage',
        'Classification', 'Creator/Contributors', 'Created on',
        'Last updated on', 'Source (of Knowledge/definition)',
        'defined by ontology', 'imported from', 'language', 'curation status'],
       dtype='object'),
 Index(['Label', 'Label German', 'Definition', 'Notes', 'Example of usage',
        'Synonyms', 'Classification', 'Creator/Contributors', 'Created on',
        'Last updated on', 'Source (of Knowledge/definition)',
        'defined by ontology', 'imported from', 'language', 'curation status'],
       dtype='object'),
 Index(['Label', 'Definition', 'Notes', 'Example of usage', 'Synonyms',
        'Classification', 'Denotation/symbol/acronym', 'Units',
        'Creator/Contributors', 'Cr

In [42]:
# All type of Columns in Excel
sorted(set([j for i in [read_excel(globals()['excel_file'+'{}'.format(i)]).columns.tolist() for i in range(1,7)] for j in i]), key=str.lower)


['Classification',
 'Created on',
 'Creator/Contributors',
 'curation status',
 'defined by ontology',
 'Definition',
 'Definition [Deutsch]',
 'Definition [English]',
 'Denotation/symbol/acronym',
 'Example of usage',
 'identifier',
 'imported from',
 'Label',
 'Label (German)',
 'Label [Deutsch]',
 'Label [English]',
 'Label German',
 'language',
 'Last updated on',
 'Notes',
 'Notes [Deutsch]',
 'Notes [English]',
 'Parent Class',
 'relevant to CQ',
 'Source (of Knowledge/definition)',
 'Synonyms',
 'Synonyms [Deutsch]',
 'Synonyms [English]',
 'Units']

In [63]:
org_to_std_dict = {'Classification': 'Classification','Created on': 'Created On', 'Creator/Contributors': 'Creator and Contributors', 
                 'curation status':'Curation Status', 
                  'defined by ontology':'Define by Ontology', 'Definition':'Definition [English]', 
                 'Definition [Deutsch]':'Definition [German]',
                 'Definition [English]':'Definition [English]', 'Denotation/symbol/acronym':'Denotation/symbol/acronym', 
                 'Example of usage':'Example of Usage',
                 'identifier': 'Identifier', 'imported from':'Imported From', 
                 'Label':'Label [English]', 'Label (German)':'Label [German]', 'Label [Deutsch]':'Label [German]', 
                 'Label [English]':'Label [English]', 
                 'Label German':'Label [German]','language':'Language', 'Last updated on': 'Last updated on', 'Notes':'Comment [English]', 
                 'Notes [Deutsch]':'Comment [German]', 'Notes [English]':'Comment [English]',
                  'Parent Class':'Parent Class', 'relevant to CQ':'Relevant to CQ', 'Source (of Knowledge/definition)':'Definition Source',
                  'Synonyms':'Synonyms [English]', 'Synonyms [Deutsch]':'Synonyms [German]', 'Synonyms [English]':'Synonyms [English]', 'Units':'Units'}
len(set(org_to_std_dict.values()))

23

In [61]:
df_luis = pd.read_csv('./csvs/digitrubber_robot_v4.csv', encoding='utf-8')

rename_col_dict = {'SubClass': 'SubClassID', 'Label':'Label [English]', 'Label.1': 'Label [German]', 
                   'Class':'Parent Class', 'Definition': 'Definition [English]', 'imported from':'Imported From',
                   'definition source': 'Definition Source', 'comments': 'Comment [English]', 
                   'Synonyms': 'Synonyms [English]', 'dateSubmitted': 'Date Submitted'}

df_luis.rename(columns=rename_col_dict, inplace=True)
print (sorted(rename_col_dict.values(), key=str.lower))
print(df_luis.columns)

df_luis['Classification'] = np.nan
df_luis['Comment [English]'] = df_luis['Comment [English]'] # Notes, Notes [English]
df_luis['Comment [German]'] = np.nan # Notes [Deutsch]
df_luis['Created On'] = np.nan
df_luis['Creator and Contributors'] = np.nan # 'Creator/Contributors'
df_luis['Curation Status'] = np.nan
df_luis['Date Submitted'] = df_luis['Date Submitted']
df_luis['Define by Ontology'] = np.nan
df_luis['Definition [English]'] = df_luis['Definition [English]'] # Definition
df_luis['Definition [German]'] = np.nan
df_luis['Definition Source'] = df_luis['Definition Source']
df_luis['Denotation/symbol/acronym'] = np.nan
df_luis ['Example of Usage'] = np.nan
df_luis['Identifier'] = np.nan
df_luis['Imported From'] = df_luis['Imported From']
df_luis['Label [English]'] = df_luis['Label [English]'] #  'Label', 'Label [English]'
df_luis['Label [German]'] = df_luis['Label [German]'] # 'Label German'
df_luis['Language'] = np.nan
df_luis['Last updated on'] = np.nan
df_luis['Parent Class'] = df_luis['Parent Class']
df_luis['Relevant to CQ'] = np.nan
df_luis['SubClassID'] = df_luis['SubClassID']
df_luis['Synonyms [English]'] = df_luis['Synonyms [English]'] # Synonyms, Synonyms [English]
df_luis['Synonyms [German]'] = np.nan # Synonyms [Deutsch]
df_luis['Units'] = np.nan

print(len(df_luis.columns), df_luis.columns)
df_luis.head(5)

['Comment [English]', 'Date Submitted', 'Definition [English]', 'Definition Source', 'Imported From', 'Label [English]', 'Label [German]', 'Parent Class', 'SubClassID', 'Synonyms [English]']
Index(['SubClassID', 'Label [English]', 'Label [German]', 'Parent Class',
       'Definition [English]', 'Imported From', 'Definition Source',
       'Comment [English]', 'Synonyms [English]', 'Date Submitted'],
      dtype='object')
25 Index(['SubClassID', 'Label [English]', 'Label [German]', 'Parent Class',
       'Definition [English]', 'Imported From', 'Definition Source',
       'Comment [English]', 'Synonyms [English]', 'Date Submitted',
       'Classification', 'Comment [German]', 'Created On',
       'Creator and Contributors', 'Curation Status', 'Define by Ontology',
       'Definition [German]', 'Denotation/symbol/acronym', 'Example of Usage',
       'Identifier', 'Language', 'Last updated on', 'Relevant to CQ',
       'Synonyms [German]', 'Units'],
      dtype='object')


,SubClassID,Label [English],Label [German],Parent Class,Definition [English],Imported From,Definition Source,Comment [English],Synonyms [English],Date Submitted,Classification,Comment [German],Created On,Creator and Contributors,Curation Status,Define by Ontology,Definition [German],Denotation/symbol/acronym,Example of Usage,Identifier,Language,Last updated on,Relevant to CQ,Synonyms [German],Units
0,ID,AL rdfs:label@en,AL rdfs:label@de,SC %,A IAO:0000115,A IAO:0000412,A IAO:0000119,AL rdfs:comment@en,A IAO:0000118,A http://purl.org/dc/terms/dateSubmitted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DIGITRUBBER:DIGITRUBBER_000001,measurement quality [imr],NaN,http://purl.obolibrary.org/obo/BFO_0000019,In relation to optical measuring methods a \nrating for the precision of a reconstructed\nmeasurement object,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DIGITRUBBER:DIGITRUBBER_000002,extrudate [imr],NaN,http://purl.obolibrary.org/obo/BFO_0000040,definded rigid Object based as result of an \nextrusion process,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DIGITRUBBER:DIGITRUBBER_000003,geometry [imr],NaN,http://purl.obolibrary.org/obo/BFO_0000019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DIGITRUBBER:DIGITRUBBER_000004,geometry [geosparql],NaN,http://purl.obolibrary.org/obo/BFO_0000019,A coherent set of direct positions in space. The positions are held within a Spatial Reference System (SRS).,http://www.opengis.net/ont/geosparql#,http://www.opengis.net/ont/geosparql#Geometry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Assign Ids to all excel labels
# Change columns-name to standard column names

def change_column_name(df):
    df.rename(columns=org_to_std_dict, inplace=True)
    return df

def assign_ids(df, from_df):
    df['SubClassID'] = np.nan
    
    return df

for i in range(1,7):
    df = read_excel(globals()['excel_file'+'{}'.format(i)])
    df = change_column_name(df)
    df = assign_ids(df, df_luis)
    df.to_csv(globals()['excel_file'+'{}_id'.format(i)], index=False, encoding='utf-8')